<a href="https://colab.research.google.com/github/yuliiabosher/Cyber_Resilience_Course/blob/main/Testing_software.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Software testing
---

When we introduce new software into a wider system we need to ensure that it won't also introduce problems such as unauthorised access, data inaccuracy, or data corruption.

Any software that is made available to users is vulnerable to misuse.  The software must be written to reduce the risk and the impact of misuse.

Software, and data, can be protected in a range of way:
*  passwords
*  user access levels
*  access control lists
*  access keys or tokens

Before software reaches end users, it should be fully tested for functionality (does it do what it needs to do) and security (can it be compromised)

## Exercise 1 - Understand the example scenario

A small app is written to store a balance in a savings account for a user.  The user must log in (to demonstrate that they are authorised to use the app) then can do one of three things:
*  view a user's balance
*  view all records
*  add funds for a user
*  remove funds for a user (funds must not go below 0)
*  add a new user with a balance of 0
*  remove a user
*  exit the app

Examine each of the code cells, and their explanatory text below.  Add extra print instructions if it helps, to get an idea of what a query looks like, what data is returned from the database, etc.

**You are going to test this code and so need to have an idea of what it does.  It has errors and your testing should find these errors.**

### Create a database (using sqlite3) in this colab's file system
---
(database connection will be created and named **db_conn**)

Run the code in the cell below, then open the files panel on the left of the screen, and click on the refresh button.  You will be able to see that the database has been created.  [Video demonstration](https://vimeo.com/900384283/c15acc62a7?share=copy)

The code:
*  creates a connection to a database with name savings.sqlite
*  creates a cursor that can perform SQL queries to  data

RUN THIS CODE **ONCE** SO THAT YOU CAN SEE WHAT IT DOES


In [1]:
import sqlite3

def create_database(db_name):
  conn = sqlite3.connect(db_name)
  cur = conn.cursor()
  query = "CREATE TABLE IF NOT EXISTS savings(user_id INTEGER PRIMARY KEY, balance REAL)"
  cur.execute(query)
  conn.commit()
  print("Database connected and savings table created if it didn't exist already")
  return conn

db_conn = create_database("/content/savings.sqlite")

Database connected and savings table created if it didn't exist already


## View the database to check it looks correct
---

Get the data from the database table and return the data and the column headings

RUN THIS CODE AS OFTEN AS YOU NEED TO - IT WILL ALWAYS SHOW WHAT DATA IS IN THE DATABASE


In [ ]:
from tabulate import tabulate

def get_data(conn):
  cur = conn.cursor()
  query = "SELECT * FROM savings"
  cur.execute(query)
  data = cur.fetchall()
  columns = tuple(map(lambda x: x[0], cur.description))
  return data, columns

table_data, columns = get_data(db_conn)
print(tabulate(table_data, headers = columns))

user_id    balance
---------  ---------


### Add a balance for a user with id 1
---
The function below will add a new record to the database with a given user ID and a starting balance of 0.

Run it **JUST ONCE** to add the record for a user with id 1

In [ ]:
def add_new_user(user_id, conn):
  cur = conn.cursor()
  query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
  cur.execute(query)
  db_conn.commit()
  print(f"User {str(user_id)} record added")

# run the add new user function to add user with id 1, then run the next cell to get the data again and print it to show that the user with id 1 has been added
add_new_user(1, db_conn)

User 1 record added


### Check that the balance was saved by getting the data, then print the whole database table
---
These functions can be used every time you need to get all the data from the database and want to display it to check that the database has the right data in it.

In [ ]:
def get_data(conn):
  cur = conn.cursor()
  query = "SELECT * FROM savings"
  cur.execute(query)
  data = cur.fetchall()
  columns = tuple(map(lambda x: x[0], cur.description))
  return data, columns

table_data, columns = get_data(db_conn)
print(tabulate(table_data, headers=columns))

  user_id    balance
---------  ---------
        1          0


## Check that a user exists before trying to do anything with their data
---

This function will check that a user with a given id exists in the database.  This helps to reduce errors that would occur if code tried to update or remove a record that doesn't exist.

The function can be used before every update or remove operation.

In [ ]:
def does_exist(user_id, conn):
  cur = conn.cursor()
  query = f"SELECT * FROM savings WHERE user_id = {str(user_id)}"
  cur.execute(query)
  data = cur.fetchone()
  return data is not None

# run the does_exist function to check that user id 1 exists in the database
print(does_exist(1, db_conn))

True


## Display a user's balance
---
Given a user id, display the balance for that user

In [ ]:
def get_user_data(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT balance FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchone()
    return data[0]
  else:
    return "User does not exist"

# run the get_user_data function to get the balance data for user id 1
balance = get_user_data(1, db_conn)
print(balance)

0.0


### Add funds
---

Run the code below to add 15 to the balance for user with user_id 1

In [ ]:
def add_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = db_conn.cursor()
    query = f"UPDATE savings SET balance=balance+{str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    return f"Record for user: {str(user_id)} updated - {str(amount)} added"
  else:
    return "User does not exist"

# run the add_funds function to add 15 to the balance for user id 1, then print the result, then print the table to see that it has been added
result = add_funds(1, 15, db_conn)
print(result)
table_data, columns = get_data(db_conn)
print(tabulate(table_data, headers=columns))

Record for user: 1 updated - 15 added
  user_id    balance
---------  ---------
        1         15


### Remove funds
---

Run the function below to remove 15 from the balance for user with user_id 1.


In [ ]:
def remove_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance - {str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    return f"Record for user: {str(user_id)} updated - {str(amount)} removed"
  else:
    return "User does not exist"

# run the remove_funds function to remove 15 from the balance for user_id, print the result and then print the whole table to show that the balance has been removed
result = remove_funds(1, 15, db_conn)
print(result)
table_data, columns = get_data(db_conn)
print(tabulate(table_data, headers=columns))

Record for user: 1 updated - 15 removed
  user_id    balance
---------  ---------
        1          0


##  Exercise 2 - Have a go
---
Write a function (call it ```add_user_set()``` that will call the add_new_user(user_id) function to add users with ids from 2 to 10.  

**Hint**:  *use a for loop to count from 2 to 10 and add a new user with the count as its id each time*

In [ ]:
def add_user_set():
  for id in range(2,11):
    add_new_user()
table_data, columns = get_data(db_conn)
print(tabulate(table_data, headers=columns))

  user_id    balance
---------  ---------
        1          0


## Exercise 3 - write a function to remove a user
---

Write a final function, called remove_user(user_id) that will remove an existing user.  Remember to check that the user exists before removing.  (Base your code on adding a new user but change the query as shown below).  

The SQL query for removing a specified `user_id` is:  
```query = "DELETE FROM savings WHERE user_id={str(user_id)}"```



In [ ]:
def remove_new_user(user_id, conn):
  cur = conn.cursor()
  query = f"DELETE FROM savings WHERE user_id={str(user_id)}"
  cur.execute(query)
  db_conn.commit()
  print(f"User {str(user_id)} record removed")

remove_new_user(1, db_conn)
table_data, columns = get_data(db_conn)
print(tabulate(table_data, headers=columns))

User 1 record removed
user_id    balance
---------  ---------


## Exit the app, closing the database connection
---
All apps that access databases should close the connection on exiting.  This is generally done when a set of processing has been completed and uses the instruction:  

```db_conn.close()```

where db_conn is the name of the connection to the database.

In [ ]:
# close and nullify the database connection
def exit_app(conn):
  conn.close()
  conn = None
  print("Exiting app...")
  return conn

db_conn = exit_app(db_conn)

Exiting app...


# Exercise 4 - Create a test plan to test the functionality of the example app
---

## 1.  Functionality testing

Now that you have a basic (at least) understanding of the code for this database app, and you have a database created (savings.sqlite) with 9 records already added, devise a testing plan to cover the functionality.

The objective of the testing plan is to make the app fail so that you can identify all the areas that need to be improved to make it robust.

The code in the next cell is the entire app (excluding the creation of the database which has already been done).

Run the code to make the functions active.

In [2]:
import sqlite3
from tabulate import tabulate

# gets a copy of the data from the savings table
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_data(conn):
  cur = conn.cursor()
  query = "SELECT * FROM savings"
  cur.execute(query)
  data = cur.fetchall()
  columns = tuple(map(lambda x: x[0], cur.description))
  return data, columns

# prints the balance for.   The table will have a number of rows corresponding to the number of users plus a header row.
# it will be possible to get a list of users from this table
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def get_user_data(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"SELECT balance FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchone()
    return data[0]
  else:
    return "User does not exist"

# check that a user with a given id does or doesn't exist in the savings table
# returns True or False
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def does_exist(user_id, conn):
  cur = conn.cursor()
  query = f"SELECT * FROM savings WHERE user_id = {str(user_id)}"
  cur.execute(query)
  data = cur.fetchone()
  return data is not None

# adds a user with the given id to the savings table
# returns a message to confirm that this has happened
# ---------------------------------------------------------------------------------------------------------------------------------------------------
def add_new_user(user_id, conn):
  cur = conn.cursor()
  query = f"INSERT INTO savings(user_id,balance) VALUES({str(user_id)},0)"
  cur.execute(query)
  conn.commit()
  print(f"User {str(user_id)} record added")
  return True

# removes a user with the given id from the savings table after checking that the user does exist
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def remove_user(user_id, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"DELETE FROM savings WHERE user_id={str(user_id)}"
    cur.execute(query)
    print(f"User {str(user_id)} deleted")
    return True
  else:
    print("User does not exist")
    return False

# updates the balance for the given user by adding the given amount onto the balance
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def add_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance+{str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    print(f"Record for user: {str(user_id)} updated - {str(amount)} added")
    return True
  else:
    print("User does not exist")
    return False

# updates the balance for the given user by subtracting the given amount from the balance
# returns a message to confirm that this has happened or that the user doesn't exist
# ----------------------------------------------------------------------------------------------------------------------------------
def remove_funds(user_id, amount, conn):
  if does_exist(user_id, conn):
    cur = conn.cursor()
    query = f"UPDATE savings SET balance=balance - {str(amount)} WHERE user_id={str(user_id)}"
    cur.execute(query)
    data = cur.fetchall()
    print(f"Record for user: {str(user_id)} updated - {str(amount)} removed")
    return True
  else:
    print("User does not exist")
    return False

# nullifies the connection so that is can't accidentally be used again
# ----------------------------------------------------------------------------------------------------------------------------------
def exit_app(conn):
  conn.close()
  conn = None
  print("Exiting app...")
  return conn

The required functionality is:  
* view the balance for a given user
* view all records in the savings table
* add funds for a user
* remove funds for a user (funds must not go below 0)
* add a new user with a balance of 0
* remove a user
* exit the app



### Prepare a new database ready for testing
---

**FIRST** - delete the file savings.sqlite from this colab's file system (open the file panel, delete the file by clicking on the three dots next to the file name, then select *Delete file*)


**THEN** run the code below to recreate the database and table ready for testing.

In [3]:
#  APP CODE
# create the database, add 10 users with starting balances from the list, display the table and exit the app - this code will set up the databse ready for testing.


def setup_database(db_conn):
  starting_balances = [44,5,42,32,25,5,26,39,45,29]
  for id in range(1, 11):
      add_new_user(id, db_conn)
      add_funds(id, starting_balances[id-1], db_conn)
  print("test data added")

!!rm /content/savings.sqlite
db_conn = create_database("/content/savings.sqlite")
setup_database(db_conn)
data, columns = get_data(db_conn)
print(tabulate(data, headers = columns))
exit_app(db_conn)

Database connected and savings table created if it didn't exist already
User 1 record added
Record for user: 1 updated - 44 added
User 2 record added
Record for user: 2 updated - 5 added
User 3 record added
Record for user: 3 updated - 42 added
User 4 record added
Record for user: 4 updated - 32 added
User 5 record added
Record for user: 5 updated - 25 added
User 6 record added
Record for user: 6 updated - 5 added
User 7 record added
Record for user: 7 updated - 26 added
User 8 record added
Record for user: 8 updated - 39 added
User 9 record added
Record for user: 9 updated - 45 added
User 10 record added
Record for user: 10 updated - 29 added
test data added
  user_id    balance
---------  ---------
        1         44
        2          5
        3         42
        4         32
        5         25
        6          5
        7         26
        8         39
        9         45
       10         29
Exiting app...


## 2.  Your task
---
Open a copy of this testing document and fill in a full set of test cases.

[Testing document](https://docs.google.com/document/d/1zSWvL5Ufx7O0IgSXcEzFqsouXGiNQ3JWelOeGRSHEIM/edit?usp=sharing)

Edit in Google drive and share your document (Anyone with the link can view) and add the link in the text box below.

**Double-click on this cell to enable editing**

Add the link to your testing plan inside the round brackets [**here**]()

### Test the app following your plan
---
Use the code cell below to add and run code to run all test cases.  Add a screenshot of any test output that doesn't match the expected output in your testing table.

Run the code in the code cell above to make sure that all the functions are available to use.

A starter test has been added for you.   Run the code in the cell.  If you don't get the expected outcome, add and remove users as needed until you get this outcome.

In [4]:
# test 1 - view all records in the database
db_conn = sqlite3.connect("/content/savings.sqlite")
data, columns = get_data(db_conn)
print(tabulate(data, headers = columns))

  user_id    balance
---------  ---------
        1         44
        2          5
        3         42
        4         32
        5         25
        6          5
        7         26
        8         39
        9         45
       10          0


In [5]:
# test 2 - view the balance for a valid user
get_user_data(1, db_conn)

44.0

In [6]:
# test 3 - view the balance for a non-valid user
get_user_data(15, db_conn)

'User does not exist'

In [7]:
# test 4 - view the balance with incorrect data type - one-digit string
get_user_data("1", db_conn)

44.0

In [8]:
# test 5 - add new user with id that already exists and correct data type - integer
add_new_user(1, db_conn)

IntegrityError: UNIQUE constraint failed: savings.user_id

In [9]:
# test 6 - add new user with id that does not exist and correct data type - integer
add_new_user(11, db_conn)

User 11 record added


True

In [13]:
# test 7 - add new user with id that already exists and incorrect data type – one-digit string
add_new_user("1", db_conn)

IntegrityError: UNIQUE constraint failed: savings.user_id

In [14]:
# test 8 - add new user with id that does not exist and incorrect data type – two-digit string
add_new_user("12", db_conn)

IntegrityError: UNIQUE constraint failed: savings.user_id

In [15]:
# test 9 - add new user with id that already exists and incorrect data type – float
add_new_user(1.0, db_conn)

IntegrityError: UNIQUE constraint failed: savings.user_id

In [16]:
# test 10 - add new user with id that does not exist and incorrect data type – float
add_new_user(13.0, db_conn)

User 13.0 record added


True

In [17]:
# test 11 - add new user with id that does not exist and incorrect data type – non-digit string
add_new_user("p", db_conn)

OperationalError: no such column: p

In [18]:
# test 12 - add funds for a user with id that exists, valid data type – integer and funds as valid data type – float
add_funds(1, 10.0, db_conn)

Record for user: 1 updated - 10.0 added


True

In [19]:
# test 13 - add funds for a user with id that does not exist, valid data type – integer and funds as valid data type – float
add_funds(14, 10.0, db_conn)

User does not exist


False

In [20]:
# test 14 - remove funds for a user with id that already exists and correct data type – integer,
# funds as correct data type – float, amount not exceeding the balance
remove_funds(2, 3.0, db_conn)

Record for user: 2 updated - 3.0 removed


True

In [21]:
# test 15 - emove funds for a user with id that already exists and correct data type – integer,
# funds as correct data type – integer, amount not exceeding the balance
remove_funds(2, 3, db_conn)

Record for user: 2 updated - 3 removed


True

In [22]:
# test 16 - emove funds for a user with id that already exists and correct data type – integer,
# funds as correct data type – integer, amount  exceeding the balance
remove_funds(3, 45, db_conn)

Record for user: 3 updated - 45 removed


True

In [23]:
# test 17 - remove a user for user with id that already exists and correct data type – integer
remove_user(1, db_conn)

User 1 deleted


True

In [24]:
# test 18 - remove a user with id that does not exist and correct data type – integer
remove_user(15, db_conn)

User does not exist


False

In [25]:
# test 19 - exit the app while the app is still open, test performed in correct order
exit_app(db_conn)

Exiting app...


In [26]:
# test 20 - exit the app while the app is closed, test performed in correct order
exit_app(db_conn)

Exiting app...
